## Dependencies

In [1]:
import warnings
import tensorflow_hub as hub
from tensorflow.keras import Model
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path_list = ['/kaggle/input/107-googleq-a-train-use-qa-text-pre-process2-3fold/model_fold_1.h5', 
                   '/kaggle/input/107-googleq-a-train-use-qa-text-pre-process2-3fold/model_fold_2.h5',
                   '/kaggle/input/107-googleq-a-train-use-qa-text-pre-process2-3fold/model_fold_3.h5']
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use-qa'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
EMBEDDDING_SIZE = 512

## Test set

In [6]:
# Test features
X_test_title = test['question_title']
X_test_body = test['question_body']
X_test_answer = test['answer']

X_test = [X_test_title, X_test_body, X_test_answer]

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

In [8]:
def model_fn():
    input_title = Input(shape=(1,), dtype=tf.string, name='input_title')
    embedding_title = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_title)

    input_body = Input(shape=(1,), dtype=tf.string, name='input_body')
    embedding_body = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_body)

    input_answer = Input(shape=(1,), dtype=tf.string, name='input_answer')
    embedding_answer = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_answer)

    x = Concatenate()([embedding_title, embedding_body, embedding_answer])
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(N_CLASS, activation='sigmoid', name='output')(x)
    model = Model(inputs=[input_title, input_body, input_answer], outputs=[output])
    
    return model

# Make predictions

In [9]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [10]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.905331,0.664378,0.347284,0.680501,0.475412,0.589105,0.620876,0.580157,0.281732,...,0.888849,0.877525,0.594197,0.940898,0.946094,0.789385,0.075847,0.082719,0.771147,0.901620
1,46,0.871425,0.574921,0.003800,0.663241,0.820404,0.911941,0.573530,0.478909,0.121197,...,0.730069,0.923584,0.620132,0.956895,0.967788,0.843719,0.927771,0.146742,0.087030,0.871465
2,70,0.882669,0.650033,0.014039,0.740855,0.869936,0.905120,0.594877,0.497210,0.314322,...,0.842555,0.899434,0.601258,0.948130,0.946005,0.805438,0.269719,0.147538,0.688334,0.869222
3,132,0.842243,0.437451,0.014003,0.711408,0.819118,0.885668,0.551620,0.483537,0.305042,...,0.685533,0.925054,0.683041,0.952984,0.973363,0.879326,0.704510,0.128913,0.698217,0.870472
4,200,0.944369,0.558940,0.010065,0.869440,0.844559,0.935435,0.627543,0.569130,0.291596,...,0.805161,0.943846,0.650252,0.975960,0.975461,0.865279,0.483930,0.207410,0.429568,0.912396


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.873915,0.569386,0.039239,0.713463,0.785962,0.847069,0.570196,0.484552,0.231068,...,0.767816,0.922463,0.646538,0.955724,0.966255,0.850954,0.545962,0.148095,0.502044,0.894354
std,2812.670060,0.049335,0.120714,0.068864,0.093083,0.095545,0.120661,0.043917,0.079678,0.121327,...,0.085389,0.028003,0.038123,0.017079,0.014906,0.039516,0.304830,0.063815,0.252463,0.023405
min,39.000000,0.708261,0.309248,0.001404,0.393785,0.410806,0.276197,0.487703,0.362344,0.042558,...,0.586362,0.814255,0.554447,0.877878,0.901080,0.704982,0.005701,0.012505,0.038339,0.791705
25%,2572.000000,0.841753,0.474018,0.005876,0.658379,0.736863,0.814183,0.534890,0.423846,0.131148,...,0.699855,0.906326,0.619731,0.946460,0.958182,0.826807,0.248025,0.104305,0.284614,0.879101
50%,5093.000000,0.875005,0.563182,0.011580,0.718342,0.795539,0.889601,0.557646,0.461467,0.203622,...,0.763124,0.927187,0.643209,0.958731,0.969825,0.856118,0.663371,0.151486,0.507885,0.895995
75%,7482.000000,0.909548,0.664606,0.036309,0.778202,0.852300,0.925938,0.605557,0.521341,0.308979,...,0.843193,0.943431,0.671862,0.967736,0.976990,0.880011,0.807761,0.191503,0.706968,0.911173
max,9640.000000,0.966760,0.837092,0.503318,0.932532,0.981270,0.979163,0.687794,0.738374,0.629329,...,0.924123,0.981537,0.759859,0.988750,0.992524,0.954140,0.941420,0.331107,0.978139,0.953153
